# Prediction Difference Computation Between Age Group

Logic: 10 Folder Prediction on self-model vs average other groups

In [1]:
%pwd

'/mnt/d/OneDrive - Kyushu University/ESG09_Article/Code'

In [2]:
%cd ..

/mnt/d/OneDrive - Kyushu University/ESG09_Article


/home/linux/anaconda3/envs/automl/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package

In [3]:
from joblib import dump, load
import numpy as np
import os 
import pandas as pd
import random
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb

## Load dataset and divide them into subgroup

In [4]:
Df_Filename = os.path.join("Data", "GallupWB_Ml64var1911k14wave_v1.parquet")

In [5]:
Df = pd.read_parquet(Df_Filename)

In [6]:
Df.shape

(1911212, 64)

In [7]:
Df.columns

Index(['wave', 'INCOME_2', 'Cantril_ladder', 'Health_disable', 'Relative_have',
       'Living_standard_change', 'Enough_food', 'Enough_shelter',
       'Well_rested', 'Respected', 'Smile', 'Interesting_thing', 'Enjoyment',
       'Physical_pain', 'Worry', 'Sadness', 'Stress', 'Anger',
       'City_satisficied', 'Economic_change', 'Goodtime_job', 'Sat_pubtran',
       'Sat_road', 'Sat_edu', 'Sat_qualityair', 'Sat_qualitywater',
       'Sat_healthcare', 'Sat_affhouse', 'Sat_oppofriend', 'Good_minorities',
       'Good_gayles', 'Good_immigrants', 'Donated', 'Volunteer',
       'Help_stranger', 'Voice_official', 'Local_police', 'Safety_walk',
       'Stolen', 'Assualted', 'Religion_importance', 'Children_respected',
       'Children_learn', 'Women_respected', 'Sat_dealpoor', 'Sat_perserveenv',
       'Freedom_chooselife', 'Conf_military', 'Conf_judicial',
       'Conf_government', 'Conf_financial', 'Conf_honestyelections',
       'Freedom_media', 'Corruption_business', 'Corruption_governm

In [8]:
Df.sort_index().head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
367305,4,1.351047e+01,6.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,21.0,1.0,1.0,0.0,2.0,1.0,USA
367306,4,2.296780e+06,6.0,2.0,1.0,-0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,34.0,1.0,1.0,0.0,2.0,5.0,USA
367307,4,3.512722e+05,8.0,2.0,1.0,-1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,44.0,1.0,1.0,0.0,2.0,5.0,USA
367308,4,5.404188e+04,8.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,67.0,1.0,1.0,0.0,1.0,5.0,USA
367309,4,0.000000e+00,8.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,49.0,2.0,1.0,0.0,1.0,1.0,USA


In [9]:
Df = Df.sample(frac=1, random_state=42)

In [10]:
Df.head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1469152,10,35602.470662,7.0,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,64.0,1.0,5.0,0.0,2.0,3.0,NZL
1562723,11,20170.734058,5.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,18.0,1.0,6.0,0.0,2.0,1.0,LBN
708695,6,8664.458086,4.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,15.0,1.0,6.0,1.0,3.0,3.0,BWA
1828455,12,30706.925977,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,33.0,2.0,1.0,1.0,2.0,1.0,KWT
968859,7,4826.805232,8.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,29.0,8.0,1.0,1.0,3.0,4.0,HND


In [11]:
Df['COUNTRY_ISO3'] = Df['COUNTRY_ISO3'].astype('category')

In [12]:
Df.isna().any().any()

False

In [13]:
Df_young = Df.loc[Df['Age']<=40, :].drop(columns=['Age'])

In [14]:
Df_middle = Df.loc[(Df['Age']>40)&(Df['Age']<=65), :].drop(columns=['Age'])

In [15]:
Df_old = Df.loc[Df['Age']>65, :].drop(columns=['Age'])

In [16]:
yy = Df_young['Cantril_ladder']
Xy = Df_young.drop(columns=['Cantril_ladder'])

In [17]:
ym = Df_middle['Cantril_ladder']
Xm = Df_middle.drop(columns=['Cantril_ladder'])

In [18]:
yo = Df_old['Cantril_ladder']
Xo = Df_old.drop(columns=['Cantril_ladder'])

## Prediction of Each Group Based on Different Models

### Young Hyperparameter Reload, build model, and conduct 10 fold 10 times CV

In [19]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [20]:
bayes_search_young = load('Results/BayesSearchAgeYoung20iter.joblib')

In [21]:
CV_result = bayes_search_young.cv_results_
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).iloc[0,:]

mean_fit_time                                                     21.459076
std_fit_time                                                       0.223054
mean_score_time                                                    0.098571
std_score_time                                                     0.001349
param_gamma                                                        6.636085
param_learning_rate                                                0.025191
param_max_delta_step                                               3.066251
param_max_depth                                                           8
param_min_child_weight                                             0.033439
param_n_estimators                                                     1136
param_reg_alpha                                                    0.001001
param_reg_lambda                                                   0.030391
param_subsample                                                    0.797189
params      

In [21]:
print(f"Best parameters: {bayes_search_young.best_params_}")
print(f"Best score: {bayes_search_young.best_score_}")

Best parameters: OrderedDict([('gamma', 6.6360850776123375), ('learning_rate', 0.02519085390684862), ('max_delta_step', 3.0662510776734475), ('max_depth', 8), ('min_child_weight', 0.03343944487838535), ('n_estimators', 1136), ('reg_alpha', 0.001000562456331363), ('reg_lambda', 0.030390787275205566), ('subsample', 0.7971894567956928)])
Best score: 0.3522986332513942


In [22]:
dict(bayes_search_young.best_params_)

{'gamma': 6.6360850776123375,
 'learning_rate': 0.02519085390684862,
 'max_delta_step': 3.0662510776734475,
 'max_depth': 8,
 'min_child_weight': 0.03343944487838535,
 'n_estimators': 1136,
 'reg_alpha': 0.001000562456331363,
 'reg_lambda': 0.030390787275205566,
 'subsample': 0.7971894567956928}

In [23]:
young_xgb = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True, **bayes_search_young.best_params_)

In [24]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [25]:
scores = cross_val_score(young_xgb, Xy, yy, cv=kf, scoring='r2')

/home/linux/anaconda3/envs/automl/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [18:03:56] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [26]:
scores

array([0.35506462, 0.34795873, 0.3542707 , 0.34943226, 0.35387255,
       0.34925824, 0.35431603, 0.35241485, 0.35253019, 0.35508866])

In [27]:
Xy.head()

,wave,INCOME_2,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,Smile,...,Corruption_business,Corruption_government,Performance_leadership,Gender_female,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1562723,11,20170.734058,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,6.0,0.0,2.0,1.0,LBN
708695,6,8664.458086,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,6.0,1.0,3.0,3.0,BWA
1828455,12,30706.925977,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,KWT
968859,7,4826.805232,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,8.0,1.0,1.0,3.0,4.0,HND
2375401,16,6999.233312,2.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,6.0,0.0,2.0,4.0,THA


In [28]:
yy.head()

1562723    5.0
708695     4.0
1828455    2.0
968859     8.0
2375401    5.0
Name: Cantril_ladder, dtype: float64

In [29]:
random.seed(42)

In [30]:
range_limit = 8192
num_samples = 10
sampled_integers = random.sample(range(range_limit), num_samples)
print(sampled_integers)

[1824, 409, 4506, 4012, 3657, 2286, 1679, 1424, 6912, 520]


In [31]:
X = Xy
y = yy
model = young_xgb

In [32]:
m_prediction_list = [np.array(ym.index), np.array(ym.to_list())]
o_prediction_list = [np.array(yo.index), np.array(yo.to_list())]
fold_predictions_array_list = []
r2_scores_list = []
r2m_scores_list = []
r2o_scores_list = []

for i, random_integer in enumerate(sampled_integers):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_integer)
    
    r2_scores = []
    r2m_scores = []
    r2o_scores = []
    fold_predictions = []
    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        this_folder = np.array([y_test.index, y_test, y_pred]).T
        fold_predictions.append(this_folder)
    
        # Middle Predict
        ym_pred = model.predict(Xm)
        m_prediction_list.append(ym_pred)
        r2m = r2_score(ym, ym_pred)
        r2m_scores.append(r2m)
        
        # Old Predict
        yo_pred = model.predict(Xo)
        o_prediction_list.append(yo_pred)
        r2o = r2_score(yo, yo_pred)
        r2o_scores.append(r2o)
    
    print(f"Mean R2: {np.mean(r2_scores)}")
    print(f"Standard Deviation of R2: {np.std(r2_scores)}")
    print(f"R2 list: {r2_scores}")
    print(f"R2 Midlle list: {r2m_scores}")
    print(f"R2 Old list: {r2o_scores}")
    
    fold_predictions_array = pd.DataFrame(np.concatenate(fold_predictions, axis = 0), columns = ['index', 'Real_y', f'Predict_y_{i}'])
    fold_predictions_array_list.append(fold_predictions_array)
    r2_scores_list.append(r2_scores)
    r2m_scores_list.append(r2m_scores)
    r2o_scores_list.append(r2o_scores)

Mean R2: 0.3522217242009007
Standard Deviation of R2: 0.0022207814425039512
R2 list: [0.3474370255166309, 0.35521655837177646, 0.35165319106158, 0.35067460747097223, 0.35138379662532826, 0.3540339081742152, 0.35241274625247265, 0.3549953897773831, 0.3509755528862025, 0.35343446587244554]
R2 Midlle list: [0.3982138540709379, 0.3983569705009914, 0.3983072309689749, 0.39848480070279435, 0.3984157591806865, 0.3982033491997027, 0.39821748333167106, 0.3983378707589823, 0.39842239401595225, 0.3982358477474731]
R2 Old list: [0.3870935913400436, 0.38710960606034583, 0.38734082977209694, 0.3867929680475657, 0.3876853131057699, 0.38768512000982025, 0.38731615701745115, 0.3873881304044483, 0.38798766429080933, 0.3872558827749033]
Mean R2: 0.3522713519389892
Standard Deviation of R2: 0.0026598541055617457
R2 list: [0.3494232206103729, 0.3481366358517979, 0.35685315233643045, 0.3519521127188906, 0.3551054290650789, 0.348838094329185, 0.3540616136691941, 0.352993025255785, 0.3532110966784241, 0.35213

In [26]:
r2_list = [0.3474370255166309, 0.35521655837177646, 0.35165319106158, 0.35067460747097223, 0.35138379662532826, 
 0.3540339081742152, 0.35241274625247265, 0.3549953897773831, 0.3509755528862025, 0.35343446587244554] + \
[0.3494232206103729, 0.3481366358517979, 0.35685315233643045, 0.3519521127188906, 0.3551054290650789, 
 0.348838094329185, 0.3540616136691941, 0.352993025255785, 0.3532110966784241, 0.35213913887473214] + \
[0.349221387042971, 0.35720611120996193, 0.3520301555158245, 0.35225432166558057, 0.354497895231104, 
 0.35327282076700717, 0.3493641847502157, 0.35494116236601114, 0.3518906662557606, 0.3494239726250914] + \
[0.35315798838763857, 0.3514499037847493, 0.3496699323459449, 0.3493751851156581, 0.3508529713515077, 
 0.35250704059555027, 0.3532325552322757, 0.3510235240282016, 0.3552069335043221, 0.35804882468346144] + \
[0.3507773076581854, 0.3515549278894119, 0.3544489536028209, 0.35148734167075724, 0.3480300035890326, 
 0.35177672946149696, 0.35082993153367836, 0.3589595527059021, 0.3496949785145984, 0.35533843389905095] + \
[0.3508730003309982, 0.3514315414758816, 0.3518613797228972, 0.3534420781692521, 0.35251459985781575, 
 0.35460202628169335, 0.34993342824663687, 0.3561859463424003, 0.3504442957719337, 0.35030795934814074] + \
[0.3547215386907129, 0.3554349443616045, 0.35006306774202967, 0.3531144864624771, 0.3505153643907061, 
 0.34901798247750315, 0.347239943655143, 0.3557402091948568, 0.3575956351040189, 0.34990370275463467] + \
[0.3521056792439913, 0.35120601967130816, 0.35493270589358317, 0.35574804593394727, 0.3497760071354864, 
 0.3481223113538583, 0.34505349564984744, 0.35448515789901325, 0.3546505736431771, 0.3560231496692724] + \
[0.3450809167138824, 0.35412830620718205, 0.35553264729299494, 0.3511342827937526, 0.35189014216941816, 
 0.3530920545871169, 0.3604189581698062, 0.3496899015527011, 0.34924922006903625, 0.3530733119032632] + \
[0.35326107576799093, 0.35324275692404417, 0.352579949380585, 0.3541793277726609, 0.34916056752912306, 
 0.3531711366792457, 0.35259178722807427, 0.35280449906926115, 0.35139663046503733, 0.3502345671324978]

In [27]:
np.mean(r2_list)

0.3522941477426516

In [28]:
np.max(r2_list)

0.3604189581698062

In [29]:
np.min(r2_list)

0.34505349564984744

In [30]:
np.std(r2_list)

0.002779956017533995

In [31]:
r2_midle_list = [0.3982138540709379, 0.3983569705009914, 0.3983072309689749, 0.39848480070279435, 0.3984157591806865, 
                 0.3982033491997027, 0.39821748333167106, 0.3983378707589823, 0.39842239401595225, 0.3982358477474731] + \
[0.39848585432027483, 0.3984342787198446, 0.39821562987720427, 0.3985875699352537, 0.3981263764780987, 
 0.3982535523683908, 0.3980793292119965, 0.3983626806258982, 0.3983380065512734, 0.39845007713422564] + \
[0.3981717954755125, 0.39816245136181716, 0.39806181818469843, 0.3982075352011857, 0.39838104082740833,
 0.3984177580110708, 0.39823950290944865, 0.3986272697174451, 0.39824640445260706, 0.39829270355874336] + \
 [0.3982903220864713, 0.39825119870274184, 0.3982370726295237, 0.39827416041337615, 0.398224160938191, 
  0.3983698131643746, 0.39847327819866873, 0.39837954523100716, 0.3985604859872194, 0.3983655729353579] + \
[0.39856171002178864, 0.3984719566600462, 0.3985184269893507, 0.3981807950555226, 0.39824387635965486, 
 0.39844601023487725, 0.39841777527528077, 0.3983585182189797, 0.3979696583405321, 0.39815923992012003] + \
[0.3979705449160055, 0.39818320639218807, 0.3981965041175939, 0.3981027107884363, 0.3984268147742911, 
 0.3984912855490016, 0.398486844490184, 0.3982648267775315, 0.398401481956016, 0.398269492323313] + \
[0.3983767109967854, 0.3985499563081919, 0.39837133935559843, 0.3980968393525497, 0.39826308594176385, 
 0.3979965967885989, 0.39842038620980835, 0.39829287715222494, 0.3981594362844856, 0.3984316215868281] + \
[0.3981538008678903, 0.3981556523055373, 0.3982302383450812, 0.39835339617984067, 0.39796731301714716, 
 0.3984384997931625, 0.39856826799786516, 0.39836575539420627, 0.39847577647641497, 0.3985139410632047] + \
[0.3986087367254646, 0.39817519272476265, 0.3984178860648968, 0.398437567902192, 0.3981132039815042, 
 0.39833091540161214, 0.3983435359527714, 0.3984035937174498, 0.3983348413213399, 0.39825664969608887] + \
[0.39824948441751884, 0.3985380235665955, 0.39821332379600216, 0.39833412936550683, 0.39826969607598817, 
 0.3985864716753619, 0.3986024196103304, 0.3983285374134016, 0.39813269199444434, 0.39836473874429257]

In [32]:
np.mean(r2_midle_list)

0.39832107616412915

In [33]:
np.max(r2_midle_list)

0.3986272697174451

In [34]:
np.min(r2_midle_list)

0.39796731301714716

In [35]:
np.std(r2_midle_list)

0.0001525938184977891

In [36]:
r2_old_list = [0.3870935913400436, 0.38710960606034583, 0.38734082977209694, 0.3867929680475657, 0.3876853131057699, 
               0.38768512000982025, 0.38731615701745115, 0.3873881304044483, 0.38798766429080933, 0.3872558827749033] + \
[0.38773033021909065, 0.3880910657722827, 0.38682561965672346, 0.3875457579768352, 0.3871202350416171, 
 0.38745823024075776, 0.38729002584454486, 0.38747799893546087, 0.38674264605511177, 0.3872409665283052] + \
[0.38715957259272504, 0.38744725464797514, 0.3872555293889136, 0.387247627653451, 0.3872545424968573, 
 0.3869493697441523, 0.38755649264008507, 0.38737602183293685, 0.38717552228307384, 0.3879408171218277] + \
[0.3874330730310137, 0.3871958636436874, 0.38716193847574343, 0.38779685708194434, 0.3876898639888823, 
 0.38683546389199774, 0.388133012847077, 0.38675167521969256, 0.38721170955919815, 0.38701250173828816] + \
[0.38746217688731754, 0.3875387150666879, 0.3870386514190616, 0.3877213869377879, 0.38782053580063114, 
 0.3881944434321227, 0.387045345720445, 0.3868178923579615, 0.38779551679952307, 0.3866421774881533] + \
[0.3870684062123082, 0.3871456646971825, 0.38736097281493354, 0.3874399769696706, 0.38722278137586497, 
 0.38670865778800834, 0.38679654094396054, 0.38766537242234445, 0.38759480096610366, 0.3874074429788382] + \
[0.3877087209798066, 0.3875273839216312, 0.3879220236557841, 0.38677538097541786, 0.3871082799415507, 
 0.3871533271321326, 0.38734961531629164, 0.3875365026044959, 0.38726247489493537, 0.3872127261133581] + \
[0.387650729074528, 0.3874621095542875, 0.3872513901136675, 0.3875964270438006, 0.38701217979928704, 
 0.387339421380995, 0.3873213662031346, 0.38729622239025674, 0.3874823283990654, 0.38754606672976855] + \
[0.3882255771243781, 0.386649574469215, 0.38792862131473127, 0.3882508522609438, 0.38600989504052996, 
 0.3872508206897134, 0.3871762271607171, 0.3877562482863742, 0.38709152285274895, 0.3874641577770578] + \
[0.3877493597992776, 0.3873391437708307, 0.38733087396174215, 0.38721784062780473, 0.38764019778182257, 
 0.3873252087640122, 0.38773774120179805, 0.38807451790022374, 0.38709342101223676, 0.3870843516713236]

In [37]:
np.mean(r2_old_list)

0.3873616113774409

In [38]:
np.max(r2_old_list)

0.3882508522609438

In [39]:
np.min(r2_old_list)

0.38600989504052996

In [40]:
np.std(r2_old_list)

0.00038914839255171067

### Young Model Middle Prediction

Prediction is average of 10 fold * 10 times

In [33]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
middle_preidiction_df = pd.DataFrame(np.array(m_prediction_list).T, columns = column_list)

In [34]:
middle_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1469152.0,7.0,6.400946,6.555736,6.466240,6.440830,6.466629,6.285450,6.507310,6.292445,...,6.539925,6.556092,6.409082,6.523125,6.426798,6.348955,6.437340,6.451184,6.497902,6.423701
1,831522.0,5.0,3.384124,3.360096,3.413772,3.442470,3.163432,3.637131,3.342038,3.245346,...,3.383887,3.331268,3.424055,3.325405,3.423450,3.302390,3.548165,3.321219,3.316379,3.297112
2,1938251.0,2.0,4.370151,4.441043,4.409160,4.503329,4.556055,4.426740,4.470988,4.186167,...,4.389811,4.546682,4.543580,4.243330,4.468473,4.254834,4.477594,4.339351,4.316685,4.351960
3,1836183.0,10.0,7.194324,7.140410,7.165401,7.259960,7.176110,7.122205,7.076654,7.207380,...,7.215580,7.179533,7.067491,7.226800,7.159401,7.181217,7.148418,7.163084,7.195183,7.212948
4,1653464.0,3.0,6.498404,6.622296,6.525658,6.538016,6.547456,6.663548,6.587179,6.474920,...,6.398673,6.609694,6.371307,6.800985,6.451802,6.665865,6.332347,6.701022,6.627069,6.530715


In [35]:
middle_preidiction_df['Predict_y'] = middle_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [36]:
middle_preidiction_df['Predict_y'].head()

0    6.447419
1    3.354689
2    4.419019
3    7.147924
4    6.521834
Name: Predict_y, dtype: float64

In [37]:
middle_preidiction_df = middle_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [38]:
middle_preidiction_df['index'] = middle_preidiction_df['index'].astype(int)

In [39]:
middle_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1469152,7.0,6.447419
1,831522,5.0,3.354689
2,1938251,2.0,4.419019
3,1836183,10.0,7.147924
4,1653464,3.0,6.521834
5,2335499,7.0,7.453140
6,1538044,8.0,8.491811
7,1461202,9.0,8.043764
8,727466,5.0,4.685451
9,938061,3.0,3.657958


In [40]:
middle_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbYoungModelMiddlePrediction_v1.parquet'))

### Young Model Old Prediction

In [41]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
old_preidiction_df = pd.DataFrame(np.array(o_prediction_list).T, columns = column_list)

In [42]:
old_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1535061.0,2.0,4.761735,4.758277,4.452856,4.660881,4.595292,4.530033,4.545422,4.778203,...,4.891882,4.592116,4.753959,4.724636,4.918623,4.464150,4.527631,4.742477,4.569503,4.847426
1,718598.0,5.0,5.514935,5.453190,5.533300,5.374983,5.550637,5.436326,5.510552,5.432149,...,5.441208,5.577156,5.434406,5.630925,5.574893,5.630754,5.551843,5.547777,5.492942,5.303827
2,1701982.0,7.0,6.742113,6.864552,6.855124,6.769381,6.764365,6.727232,6.694466,6.730587,...,6.848044,6.857678,6.818696,6.708043,6.834156,6.784311,6.839764,6.884028,6.701709,6.759612
3,1578430.0,7.0,6.354822,6.673211,6.392900,6.593334,6.498292,6.524296,6.214782,6.468731,...,6.459635,6.563035,6.616180,6.507893,6.601959,6.591118,6.510722,6.523704,6.603150,6.591974
4,1398218.0,10.0,7.484743,7.578883,7.371109,7.423291,7.391529,7.396041,7.561340,7.576428,...,7.369772,7.561635,7.448269,7.413283,7.467255,7.506205,7.275626,7.363591,7.544077,7.408568


In [43]:
old_preidiction_df['Predict_y'] = old_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [44]:
old_preidiction_df['Predict_y'].head()

0    4.697998
1    5.481435
2    6.793449
3    6.506651
4    7.434296
Name: Predict_y, dtype: float64

In [45]:
old_preidiction_df = old_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [46]:
old_preidiction_df['index'] = old_preidiction_df['index'].astype(int)

In [47]:
old_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1535061,2.0,4.697998
1,718598,5.0,5.481435
2,1701982,7.0,6.793449
3,1578430,7.0,6.506651
4,1398218,10.0,7.434296
5,1581052,5.0,5.276537
6,1764291,7.0,4.615627
7,1322632,7.0,4.694141
8,2573377,8.0,5.338784
9,865858,8.0,6.836610


In [48]:
old_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbYoungModelOldPrediction_v1.parquet'))

### Young Model Young Prediction

In [49]:
merged_young_prediction_df = None
for young_prediction_df in fold_predictions_array_list:
    young_prediction_df['index'] = young_prediction_df['index'].astype(int)
    young_prediction_df['Real_y'] = young_prediction_df['Real_y'].astype(int)
    print(young_prediction_df.columns)
    if merged_young_prediction_df is None:
        merged_young_prediction_df = young_prediction_df
    else:
        merged_young_prediction_df = pd.merge(merged_young_prediction_df, young_prediction_df, on=['index', 'Real_y'])

Index(['index', 'Real_y', 'Predict_y_0'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_1'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_2'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_3'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_4'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_5'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_6'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_7'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_8'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_9'], dtype='object')


In [50]:
merged_young_prediction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,Predict_y_8,Predict_y_9
0,1976009,7,5.585835,5.606697,5.452052,5.622301,5.705790,5.725687,5.724606,5.699248,5.787541,5.813135
1,1036932,3,3.872958,3.693524,3.899238,3.863476,3.777722,3.834348,4.067349,3.965018,3.821776,3.866279
2,2505821,4,3.861683,3.618260,3.698452,4.012042,3.893312,4.019055,3.826325,3.761884,3.856846,4.032545
3,1135079,1,3.987695,4.035850,3.972183,4.160449,4.218924,4.020698,4.099977,3.989823,4.140359,4.217546
4,2386018,6,7.888861,7.881535,7.829309,7.823808,7.898308,7.729335,7.905879,7.831308,7.908490,7.885075


In [51]:
merged_young_prediction_df['Predict_y'] = merged_young_prediction_df.iloc[:,2:13].mean(axis = 1)

In [52]:
merged_young_prediction_df = merged_young_prediction_df[['index', 'Real_y', 'Predict_y']]

In [53]:
merged_young_prediction_df.head(10)

,index,Real_y,Predict_y
0,1976009,7,5.672289
1,1036932,3,3.866169
2,2505821,4,3.858040
3,1135079,1,4.084350
4,2386018,6,7.858191
5,1493327,5,4.763505
6,549924,1,5.829806
7,2186766,7,6.495862
8,1936498,4,4.671191
9,2235916,7,4.416826


In [54]:
merged_young_prediction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbYoungModelYoundPrediction_v1.parquet'))

### Middle Hyperparameter Reload, build model, and conduct 10 fold 10 times CV

In [55]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [22]:
bayes_search_young = load('Results/BayesSearchAgeMiddle20iter.joblib')

In [23]:
CV_result = bayes_search_young.cv_results_
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).iloc[0,:]

mean_fit_time                                                    260.862294
std_fit_time                                                       1.711591
mean_score_time                                                    0.424207
std_score_time                                                     0.010861
param_gamma                                                        0.059651
param_learning_rate                                                 0.01615
param_max_delta_step                                               0.100424
param_max_depth                                                          13
param_min_child_weight                                             1.762463
param_n_estimators                                                     3304
param_reg_alpha                                                        10.0
param_reg_lambda                                                   0.002894
param_subsample                                                    0.895621
params      

In [57]:
print(f"Best parameters: {bayes_search_young.best_params_}")
print(f"Best score: {bayes_search_young.best_score_}")

Best parameters: OrderedDict([('gamma', 0.05965055817863728), ('learning_rate', 0.016150250040658724), ('max_delta_step', 0.10042448556024319), ('max_depth', 13), ('min_child_weight', 1.7624628686422215), ('n_estimators', 3304), ('reg_alpha', 10.0), ('reg_lambda', 0.002893997263318717), ('subsample', 0.8956214250879957)])
Best score: 0.40948431281386427


In [58]:
dict(bayes_search_young.best_params_)

{'gamma': 0.05965055817863728,
 'learning_rate': 0.016150250040658724,
 'max_delta_step': 0.10042448556024319,
 'max_depth': 13,
 'min_child_weight': 1.7624628686422215,
 'n_estimators': 3304,
 'reg_alpha': 10.0,
 'reg_lambda': 0.002893997263318717,
 'subsample': 0.8956214250879957}

In [59]:
middle_xgb = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True, **bayes_search_young.best_params_)

In [60]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [61]:
scores = cross_val_score(young_xgb, Xm, ym, cv=kf, scoring='r2')

In [62]:
scores

array([0.40514408, 0.4122419 , 0.40431345, 0.40673967, 0.40809723,
       0.40270521, 0.40289283, 0.40250772, 0.39997671, 0.41249771])

In [63]:
Xm.head()

,wave,INCOME_2,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,Smile,...,Corruption_business,Corruption_government,Performance_leadership,Gender_female,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1469152,10,35602.470662,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,5.0,0.0,2.0,3.0,NZL
831522,6,4357.352396,1.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,2.0,6.0,0.0,4.0,4.0,SDN
1938251,13,7202.172618,2.0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,3.0,1.0,0.0,3.0,2.0,BIH
1836183,12,52643.260274,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,6.0,0.0,2.0,5.0,MLT
1653464,11,2561.527818,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,5.0,6.0,1.0,3.0,1.0,CHL


In [64]:
ym.head()

1469152     7.0
831522      5.0
1938251     2.0
1836183    10.0
1653464     3.0
Name: Cantril_ladder, dtype: float64

In [65]:
random.seed(42)

In [66]:
range_limit = 8192
num_samples = 10
sampled_integers = random.sample(range(range_limit), num_samples)
print(sampled_integers)

[1824, 409, 4506, 4012, 3657, 2286, 1679, 1424, 6912, 520]


In [67]:
X = Xm
y = ym
model = middle_xgb

In [68]:
y_prediction_list = [np.array(yy.index), np.array(yy.to_list())]
o_prediction_list = [np.array(yo.index), np.array(yo.to_list())]
fold_predictions_array_list = []
r2_scores_list = []
r2y_scores_list = []
r2o_scores_list = []

for i, random_integer in enumerate(sampled_integers):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_integer)
    
    r2_scores = []
    r2y_scores = []
    r2o_scores = []
    fold_predictions = []
    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        this_folder = np.array([y_test.index, y_test, y_pred]).T
        fold_predictions.append(this_folder)
    
        # Young Predict
        yy_pred = model.predict(Xy)
        y_prediction_list.append(yy_pred)
        r2y = r2_score(yy, yy_pred)
        r2y_scores.append(r2y)
        
        # Old Predict
        yo_pred = model.predict(Xo)
        o_prediction_list.append(yo_pred)
        r2o = r2_score(yo, yo_pred)
        r2o_scores.append(r2o)
    
    print(f"Mean R2: {np.mean(r2_scores)}")
    print(f"Standard Deviation of R2: {np.std(r2_scores)}")
    print(f"R2 list: {r2_scores}")
    print(f"R2 Young list: {r2y_scores}")
    print(f"R2 Old list: {r2o_scores}")
    
    fold_predictions_array = pd.DataFrame(np.concatenate(fold_predictions, axis = 0), columns = ['index', 'Real_y', f'Predict_y_{i}'])
    fold_predictions_array_list.append(fold_predictions_array)
    r2_scores_list.append(r2_scores)
    r2y_scores_list.append(r2y_scores)
    r2o_scores_list.append(r2o_scores)

Mean R2: 0.40735791523522974
Standard Deviation of R2: 0.003171409083690291
R2 list: [0.4080695228068072, 0.4057294946513367, 0.41049948034501726, 0.40420300272453, 0.40459087524266113, 0.4090729730433533, 0.4109361132843121, 0.4008739923052219, 0.40947858681515104, 0.41012511113390626]
R2 Young list: [0.3310407605571152, 0.3316980487270871, 0.331477521818205, 0.3312009029681884, 0.3312098059861447, 0.3310905459850747, 0.3313206107780008, 0.33177973469267574, 0.3313168017156958, 0.33159383900784756]
R2 Old list: [0.40786439309044764, 0.40755437214961654, 0.4074431834794635, 0.4077787071914424, 0.4078731339528372, 0.40783409885020416, 0.40764422478585316, 0.4080961122971404, 0.40749163825771684, 0.40794155378829566]
Mean R2: 0.4071631950551181
Standard Deviation of R2: 0.0020126869754770104
R2 list: [0.4067726436409145, 0.40423751591421264, 0.4083871529192522, 0.40899405118648613, 0.40746057801751934, 0.40753825549299416, 0.403526599123445, 0.40568725062976585, 0.4098761597613917, 0.409

In [45]:
r2_list = [0.4080695228068072, 0.4057294946513367, 0.41049948034501726, 0.40420300272453, 0.40459087524266113, 
           0.4090729730433533, 0.4109361132843121, 0.4008739923052219, 0.40947858681515104, 0.41012511113390626] + \
[0.4067726436409145, 0.40423751591421264, 0.4083871529192522, 0.40899405118648613, 0.40746057801751934, 
 0.40753825549299416, 0.403526599123445, 0.40568725062976585, 0.4098761597613917, 0.4091517438651996] + \
[0.41103977294378136, 0.40509508854123044, 0.4085999472487616, 0.4018625064007262, 0.407234279943987, 
 0.4061041725803123, 0.41311334977417047, 0.4097063090460341, 0.39998679820833394, 0.41038701190585125] + \
[0.4040444905265559, 0.4032537158500543, 0.4044107885929419, 0.4111338994569278, 0.40677441888892274, 
 0.41528964751114605, 0.4071659664716778, 0.4055813334315709, 0.40570438556789, 0.40771120484999057] + \
[0.40812217302807097, 0.4058551750183884, 0.4104058272553973, 0.40405654810487135, 0.4107786789478286,
 0.4097253987341207, 0.41051328276133203, 0.4049561461157284, 0.4036265637676951, 0.40435009667699784] + \
[0.4060522918895595, 0.4071561291119291, 0.4088962794109806, 0.40397971446354086, 0.40970356075112446, 
 0.4042559027076633, 0.40401236564065857, 0.4139365932336422, 0.40421837188162824, 0.4105878825839121] + \
[0.411589958291249, 0.4078047696666821, 0.4071165220520081, 0.4054386241638793, 0.40839586162269614, 
 0.4069857011533601, 0.40606147005240656, 0.403595690561797, 0.41023683606420536, 0.40571126424550574] + \
[0.40677238043281616, 0.4095565347591156, 0.40425109560125205, 0.4049254579702175, 0.40355753158077834, 
 0.40960429313938274, 0.41077794098048714, 0.4044223569002706, 0.40730107810987537, 0.4128672186547655] + \
[0.40853703672062347, 0.4046080295980814, 0.4054872064974948, 0.4018365335512627, 0.40983830922765174, 
 0.41304023492903996, 0.40480483120728117, 0.40592459158960603, 0.4110366427132993, 0.4049116316846938] + \
[0.4087141772726547, 0.40741655372808216, 0.4084220626246402, 0.4047287830313062, 0.40820404318627435, 
 0.40402046807573677, 0.4022180272990372, 0.4058084640704621, 0.41274321111513723, 0.41107805454595425]

In [46]:
np.mean(r2_list)

0.40724950651400477

In [47]:
np.max(r2_list)

0.41528964751114605

In [48]:
np.min(r2_list)

0.39998679820833394

In [49]:
np.std(r2_list)

0.0030702410223859347

In [50]:
r2_young_list = [0.3310407605571152, 0.3316980487270871, 0.331477521818205, 0.3312009029681884, 0.3312098059861447, 
                 0.3310905459850747, 0.3313206107780008, 0.33177973469267574, 0.3313168017156958, 0.33159383900784756] + \
 [0.33159369461733346, 0.3313534733543272, 0.3316411170492938, 0.3314261868227725, 0.33135410880519933, 
  0.3311345775928396, 0.3314816945457243, 0.3314377238146061, 0.33128425519994753, 0.3312270642240075] + \
 [0.3314344661926505, 0.3313417133920269, 0.3313316475081678, 0.3312714279356046, 0.33134862210612714, 
  0.3314259036665106, 0.33158914112021576, 0.3312256046197636, 0.3314152076865692, 0.3313549436744585] + \
 [0.3313313928737619, 0.33161228304769674, 0.33147376620992497, 0.3313103451333903, 0.33138746183730305, 
  0.3313294117029535, 0.3314312444974059, 0.3313247258325619, 0.33141393083513027, 0.3315594234650726] + \
 [0.3315631651667975, 0.3313477114756911, 0.3311605506342783, 0.33166468647879, 0.3314733619806921, 
  0.3311484482863857, 0.33153097309017554, 0.33126700444032964, 0.3311242356442913, 0.3313861104803444] + \
 [0.3311457634366536, 0.33144431170388855, 0.3315672162667913, 0.33107700990057887, 0.331605704298932, 
  0.33092958176393106, 0.3314488604454684, 0.33100463104579314, 0.3313088938284907, 0.3317663981944635] + \
 [0.33170427099928534, 0.3315996709931125, 0.33164502255206063, 0.3310121062961672, 0.33156526330222513, 
  0.3311855951273437, 0.33130854053305747, 0.3313955684948524, 0.33111050192313296, 0.3312592271503638] + \
 [0.33117347288201493, 0.3317032141246362, 0.3311346080164137, 0.3311966364183373, 0.33141926527800025, 
  0.3315052467301003, 0.33154769755756675, 0.33120861586101047, 0.3316013533799712, 0.3316071862697244] + \
 [0.3315380551481917, 0.33148737770318093, 0.33148683781862265, 0.33132724161568006, 0.33155696183714833, 
  0.3308157856646191, 0.33146252151530564, 0.3314444514090015, 0.33120507187674, 0.3313776795213017] + \
 [0.3313696423288658, 0.33139916727882324, 0.33126925477839997, 0.33147342535862123, 0.3315970825426855, 
  0.3317610266512715, 0.33106745963177253, 0.331225086946481, 0.3314862970092327, 0.33121546577426697]

In [65]:
np.mean(r2_young_list)

0.3313798870646174

In [66]:
np.max(r2_young_list)

0.33177973469267574

In [67]:
np.min(r2_young_list)

0.3308157856646191

In [68]:
np.std(r2_young_list)

0.00019577190453169108

In [55]:
r2_old_list = [0.40786439309044764, 0.40755437214961654, 0.4074431834794635, 0.4077787071914424, 0.4078731339528372, 
               0.40783409885020416, 0.40764422478585316, 0.4080961122971404, 0.40749163825771684, 0.40794155378829566] + \
 [0.4077422602552744, 0.4076642437779884, 0.40801765724366545, 0.4076583505062218, 0.40761779195419134, 
  0.40784792357425914, 0.40757491082002084, 0.4077914455918443, 0.4078435263308827, 0.4076796689798229] + \
 [0.4078514945653776, 0.4077239892947827, 0.40776867509480075, 0.40760991226561016, 0.4078011336333377, 
  0.4079192890028295, 0.4075605513149654, 0.40766542316669907, 0.40795264805368825, 0.40750458935901657] + \
 [0.40773218487778184, 0.40736951016630973, 0.4077384933602065, 0.40791296809702515, 0.40792377900745425, 
  0.4078631113139941, 0.40772968905958384, 0.40746738074377786, 0.4078266717677813, 0.4076298655978823] + \
 [0.4077972928135861, 0.40791601005296685, 0.40743195634312446, 0.40812171433666034, 0.40792796477408333, 
  0.40710919048784777, 0.40788777916337204, 0.40730434273467797, 0.4076056803694069, 0.4079817625461907] + \
 [0.4077401757933147, 0.40773072393928833, 0.4078152080399988, 0.40744395334452665, 0.40774839274051045, 
  0.4077521681545794, 0.40802247279524684, 0.40757128068766646, 0.4079487697803629, 0.4077309083821181] + \
 [0.407652575451387, 0.4076744247264825, 0.4076302721547943, 0.4078270943828842, 0.40763819471494567, 
  0.40766760378647293, 0.4078056710205863, 0.4077936750335117, 0.4077206482446518, 0.40803064221457475] + \
 [0.4077994416014177, 0.40792987644022083, 0.4075192558150099, 0.407873167732862, 0.4079517411015041, 
  0.4075829178195459, 0.4074548926316307, 0.40766577452593755, 0.40762657230688304, 0.40765293854281626] + \
 [0.40751381069087556, 0.4079815425966751, 0.40750145436396024, 0.40781187585858725, 0.4078292201772772, 
  0.40752803505910695, 0.407832322266218, 0.4076638731761477, 0.4076714809723291, 0.40733803058214446] + \
 [0.40789282024161155, 0.40769807186770857, 0.40768701780634875, 0.4079277671843645, 0.4077059941979697, 
  0.40769126356657936, 0.4075000779646485, 0.40769670043473694, 0.4076193032860521, 0.40805214412451973]

In [56]:
np.mean(r2_old_list)

0.4077273849056561

In [57]:
np.max(r2_old_list)

0.40812171433666034

In [58]:
np.min(r2_old_list)

0.40710919048784777

In [59]:
np.std(r2_old_list)

0.0001835968095482087

### Middle Model Young Prediction

Prediction is average of 10 fold * 10 times

In [69]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
young_preidiction_df = pd.DataFrame(np.array(y_prediction_list).T, columns = column_list)

In [70]:
young_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1562723.0,5.0,4.780888,4.519880,4.772003,4.768342,4.783091,4.933635,4.810184,4.741272,...,4.686022,4.801900,4.738563,4.680453,4.895226,4.851616,4.822000,4.792110,4.595822,4.778974
1,708695.0,4.0,4.212075,4.289929,4.382177,3.999223,4.256259,4.260980,4.266835,4.284914,...,4.114879,4.196670,4.166576,4.110175,4.516248,3.974162,4.188296,4.357482,3.982884,4.054521
2,1828455.0,2.0,6.493051,6.344342,6.421969,6.475831,6.440760,6.604690,6.430300,6.117033,...,6.402391,6.401849,6.286951,6.402671,6.408615,6.331493,6.483055,6.409377,6.627054,6.412904
3,968859.0,8.0,5.289126,5.464239,5.286373,5.247552,5.444459,5.510852,5.523960,5.511659,...,5.295139,5.344650,5.372141,5.329281,5.544105,5.185625,5.416097,5.303905,5.435267,5.549560
4,2375401.0,5.0,4.595544,4.643282,4.653045,4.494058,4.547693,4.461103,4.526562,4.616069,...,4.445099,4.719227,4.639669,4.562653,4.618272,4.627410,4.757525,4.497378,4.545774,4.607549


In [71]:
young_preidiction_df['Predict_y'] = young_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [72]:
young_preidiction_df['Predict_y'].head()

0    4.739254
1    4.177796
2    6.406743
3    5.386602
4    4.562044
Name: Predict_y, dtype: float64

In [73]:
young_preidiction_df = young_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [74]:
young_preidiction_df['index'] = young_preidiction_df['index'].astype(int)

In [75]:
young_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1562723,5.0,4.739254
1,708695,4.0,4.177796
2,1828455,2.0,6.406743
3,968859,8.0,5.386602
4,2375401,5.0,4.562044
5,723699,6.0,6.309007
6,2069375,2.0,2.571380
7,1655669,10.0,4.477426
8,776224,7.0,5.382874
9,1002429,5.0,6.740423


In [76]:
young_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbMiddleModelYoungPrediction_v1.parquet'))

### Middle Model Old Prediction

In [77]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
old_preidiction_df = pd.DataFrame(np.array(o_prediction_list).T, columns = column_list)

In [78]:
old_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1535061.0,2.0,3.969106,3.978705,4.002112,3.892348,3.855026,3.942824,3.897498,3.878018,...,3.960808,3.878315,3.774483,3.760675,4.049340,3.898403,3.829437,4.035239,3.865960,3.857313
1,718598.0,5.0,5.822968,6.024619,6.085072,5.899174,5.863245,5.779330,5.687943,5.881076,...,5.967302,5.660300,5.966939,5.996565,6.010541,5.952579,5.999315,5.890594,5.734946,5.913235
2,1701982.0,7.0,6.727386,6.814416,6.813144,6.817902,6.723922,6.719148,6.776139,6.818443,...,6.825069,6.825710,6.869914,6.748897,6.754500,6.878654,6.824768,6.674596,6.822055,6.818986
3,1578430.0,7.0,6.438377,6.546244,6.500910,6.397367,6.413377,6.568873,6.554882,6.569552,...,6.537684,6.414593,6.605969,6.445230,6.636500,6.388546,6.364561,6.403244,6.428311,6.443772
4,1398218.0,10.0,7.454511,7.423173,7.347449,7.332770,7.404515,7.324477,7.453548,7.346744,...,7.346906,7.548555,7.263584,7.338118,7.462066,7.430343,7.313087,7.283575,7.534380,7.387546


In [79]:
old_preidiction_df['Predict_y'] = old_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [80]:
old_preidiction_df['Predict_y'].head()

0    3.916924
1    5.893912
2    6.797334
3    6.478603
4    7.409482
Name: Predict_y, dtype: float64

In [81]:
old_preidiction_df = old_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [82]:
old_preidiction_df['index'] = old_preidiction_df['index'].astype(int)

In [83]:
old_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1535061,2.0,3.916924
1,718598,5.0,5.893912
2,1701982,7.0,6.797334
3,1578430,7.0,6.478603
4,1398218,10.0,7.409482
5,1581052,5.0,4.480478
6,1764291,7.0,5.359951
7,1322632,7.0,5.057159
8,2573377,8.0,4.907039
9,865858,8.0,7.000365


In [84]:
old_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbMiddleModelOldPrediction_v1.parquet'))

### Middle Model Middle Prediction

In [85]:
merged_middle_prediction_df = None
for middle_prediction_df in fold_predictions_array_list:
    middle_prediction_df['index'] = middle_prediction_df['index'].astype(int)
    middle_prediction_df['Real_y'] = middle_prediction_df['Real_y'].astype(int)
    print(middle_prediction_df.columns)
    if merged_middle_prediction_df is None:
        merged_middle_prediction_df = middle_prediction_df
    else:
        merged_middle_prediction_df = pd.merge(merged_middle_prediction_df, middle_prediction_df, on=['index', 'Real_y'])

Index(['index', 'Real_y', 'Predict_y_0'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_1'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_2'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_3'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_4'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_5'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_6'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_7'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_8'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_9'], dtype='object')


In [86]:
merged_middle_prediction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,Predict_y_8,Predict_y_9
0,1469152,7,6.578477,6.417754,6.532204,6.383023,6.512581,6.364153,6.606808,6.389546,6.540977,6.587455
1,1761492,5,5.936441,6.118467,6.045741,6.127396,6.048608,6.127149,6.125085,6.146679,6.175223,6.074225
2,1605988,6,4.549599,4.594558,4.297661,4.596976,4.715903,4.599617,4.431871,4.495641,4.724684,4.320829
3,1027429,7,6.008943,6.100342,6.119356,6.058822,6.040822,6.097470,5.986335,6.019828,6.010950,6.184084
4,2225558,5,4.788544,4.693890,4.864774,4.808639,4.688700,4.888168,4.841007,4.842601,4.956825,4.772230


In [87]:
merged_middle_prediction_df['Predict_y'] = merged_middle_prediction_df.iloc[:,2:13].mean(axis = 1)

In [88]:
merged_middle_prediction_df = merged_middle_prediction_df[['index', 'Real_y', 'Predict_y']]

In [89]:
merged_middle_prediction_df.head(10)

,index,Real_y,Predict_y
0,1469152,7,6.491298
1,1761492,5,6.092501
2,1605988,6,4.532734
3,1027429,7,6.062695
4,2225558,5,4.814538
5,1761622,3,4.233992
6,2438374,4,3.697439
7,2482997,5,6.814744
8,537644,8,7.714562
9,618292,5,4.451225


In [90]:
merged_middle_prediction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbMiddleModelMiddlePrediction_v1.parquet'))

### Old Hyperparameter Reload, build model, and conduct 10 fold 10 times CV

In [91]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [24]:
bayes_search_young = load('Results/BayesSearchAgeOld20iter.joblib')

In [25]:
CV_result = bayes_search_young.cv_results_
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).iloc[0,:]

mean_fit_time                                                     34.022171
std_fit_time                                                       0.825355
mean_score_time                                                    0.056066
std_score_time                                                     0.001606
param_gamma                                                        1.204548
param_learning_rate                                                0.030685
param_max_delta_step                                               0.380578
param_max_depth                                                          13
param_min_child_weight                                             0.002362
param_n_estimators                                                      540
param_reg_alpha                                                    6.590411
param_reg_lambda                                                   0.016427
param_subsample                                                    0.557202
params      

In [93]:
print(f"Best parameters: {bayes_search_young.best_params_}")
print(f"Best score: {bayes_search_young.best_score_}")

Best parameters: OrderedDict([('gamma', 1.2045479993021058), ('learning_rate', 0.030685320506680522), ('max_delta_step', 0.3805781770349062), ('max_depth', 13), ('min_child_weight', 0.002361695637610135), ('n_estimators', 540), ('reg_alpha', 6.59041086695112), ('reg_lambda', 0.01642699361057668), ('subsample', 0.5572023993404673)])
Best score: 0.40462796837403414


In [94]:
dict(bayes_search_young.best_params_)

{'gamma': 1.2045479993021058,
 'learning_rate': 0.030685320506680522,
 'max_delta_step': 0.3805781770349062,
 'max_depth': 13,
 'min_child_weight': 0.002361695637610135,
 'n_estimators': 540,
 'reg_alpha': 6.59041086695112,
 'reg_lambda': 0.01642699361057668,
 'subsample': 0.5572023993404673}

In [95]:
old_xgb = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True, **bayes_search_young.best_params_)

In [96]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [97]:
scores = cross_val_score(young_xgb, Xo, yo, cv=kf, scoring='r2')

In [98]:
scores

array([0.40170052, 0.39842216, 0.4012616 , 0.40491502, 0.40255832,
       0.40758486, 0.39560507, 0.40361746, 0.40749449, 0.39730465])

In [99]:
Xo.head()

,wave,INCOME_2,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,Smile,...,Corruption_business,Corruption_government,Performance_leadership,Gender_female,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1535061,10,4467.534252,1.0,1.0,-0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,5.0,6.0,0.0,3.0,3.0,MNE
718598,6,9792.918107,2.0,1.0,-0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,5.0,6.0,0.0,2.0,4.0,BLR
1701982,11,13238.653845,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.0,6.0,0.0,2.0,4.0,URY
1578430,11,10734.066166,1.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,5.0,6.0,0.0,2.0,3.0,CZE
1398218,9,24336.227568,2.0,1.0,-0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,2.0,6.0,0.0,2.0,2.0,FIN


In [100]:
yo.head()

1535061     2.0
718598      5.0
1701982     7.0
1578430     7.0
1398218    10.0
Name: Cantril_ladder, dtype: float64

In [101]:
random.seed(42)

In [102]:
range_limit = 8192
num_samples = 10
sampled_integers = random.sample(range(range_limit), num_samples)
print(sampled_integers)

[1824, 409, 4506, 4012, 3657, 2286, 1679, 1424, 6912, 520]


In [103]:
X = Xo
y = yo
model = old_xgb

In [104]:
y_prediction_list = [np.array(yy.index), np.array(yy.to_list())]
m_prediction_list = [np.array(ym.index), np.array(ym.to_list())]
fold_predictions_array_list = []
r2_scores_list = []
r2y_scores_list = []
r2m_scores_list = []

for i, random_integer in enumerate(sampled_integers):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_integer)
    
    r2_scores = []
    r2y_scores = []
    r2m_scores = []
    fold_predictions = []
    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        this_folder = np.array([y_test.index, y_test, y_pred]).T
        fold_predictions.append(this_folder)
    
        # Young Predict
        yy_pred = model.predict(Xy)
        y_prediction_list.append(yy_pred)
        r2y = r2_score(yy, yy_pred)
        r2y_scores.append(r2y)
        
        # Middle Predict
        ym_pred = model.predict(Xm)
        m_prediction_list.append(ym_pred)
        r2m = r2_score(ym, ym_pred)
        r2m_scores.append(r2m)
    
    print(f"Mean R2: {np.mean(r2_scores)}")
    print(f"Standard Deviation of R2: {np.std(r2_scores)}")
    print(f"R2 list: {r2_scores}")
    print(f"R2 Young list: {r2y_scores}")
    print(f"R2 Middle list: {r2m_scores}")
    
    fold_predictions_array = pd.DataFrame(np.concatenate(fold_predictions, axis = 0), columns = ['index', 'Real_y', f'Predict_y_{i}'])
    fold_predictions_array_list.append(fold_predictions_array)
    r2_scores_list.append(r2_scores)
    r2y_scores_list.append(r2y_scores)
    r2m_scores_list.append(r2m_scores)

Mean R2: 0.4044587694230886
Standard Deviation of R2: 0.006288313207473653
R2 list: [0.40928855258277574, 0.39222826121218857, 0.41166991315215784, 0.40850282058733733, 0.40500514081728944, 0.4017740900815038, 0.4036665994833275, 0.39548015237970746, 0.4044256549555284, 0.4125465089790695]
R2 Young list: [0.2979886627866627, 0.29791966173551165, 0.29894049798372246, 0.2979187079704628, 0.29677522905717924, 0.2982929078460792, 0.297470635886913, 0.2981276880765539, 0.29752941801220034, 0.2974326126343281]
R2 Middle list: [0.3763524808835307, 0.3764913869783597, 0.37686628596614913, 0.3762348644905381, 0.3757916163520967, 0.3764627504577214, 0.37614462200025434, 0.3767538282211552, 0.3767616103822383, 0.37620457769262705]
Mean R2: 0.4041501660956114
Standard Deviation of R2: 0.005108994427981802
R2 list: [0.40090970887714383, 0.41054186821886496, 0.3970947368501335, 0.40838961267102736, 0.3991880086739322, 0.40480922321630053, 0.39875702108856637, 0.41253052235674903, 0.4017200967047192,

In [60]:
r2_list = [0.40928855258277574, 0.39222826121218857, 0.41166991315215784, 0.40850282058733733, 0.40500514081728944, 
           0.4017740900815038, 0.4036665994833275, 0.39548015237970746, 0.4044256549555284, 0.4125465089790695] + \
 [0.40090970887714383, 0.41054186821886496, 0.3970947368501335, 0.40838961267102736, 0.3991880086739322, 
  0.40480922321630053, 0.39875702108856637, 0.41253052235674903, 0.4017200967047192, 0.40756086229867705] + \
 [0.40202573442332346, 0.40279890156678155, 0.40412586897405534, 0.4132567262296458, 0.4057563534889602, 
  0.40030292663461, 0.3991528242750305, 0.4107062552959, 0.39914742244161916, 0.4046565837655818] + \
 [0.40099112374845824, 0.4048093802140553, 0.4092575404455725, 0.4019445772993161, 0.4034955483929462, 
  0.40819749437416075, 0.4090178189516086, 0.3981720898433935, 0.40137748938103446, 0.40499312785337105] + \
 [0.40910346008949205, 0.4063619604105697, 0.3993060201042725, 0.4019420571259369, 0.405059551470204, 
  0.40504169486129304, 0.40580231966968316, 0.4043779324177812, 0.3994409528639564, 0.40967061998416543] + \
 [0.3997582993077877, 0.4062668768884481, 0.40736022219312873, 0.4152712135536001, 0.4047889673316988, 
  0.3987194017744947, 0.4020750729961864, 0.39758152273479597, 0.4019713726537125, 0.40522435270700063] + \
 [0.40244193665479333, 0.4035756525340157, 0.41203555565863803, 0.4089253645045926, 0.40545647660381856, 
  0.40951700664419977, 0.4023383816011902, 0.4042981485819209, 0.4046535203681664, 0.3970813566485354] + \
 [0.4088915352667748, 0.3917029236852765, 0.4048667493692265, 0.4076734203516684, 0.3965209201193246, 
  0.40630438342071507, 0.4001140567997533, 0.41332521762874774, 0.41689655915741264, 0.400106224619147] + \
 [0.3976099455175389, 0.39723127544533554, 0.4094101520122472, 0.4021380447715306, 0.40703840169887207, 
  0.4091624533178486, 0.40177760072439206, 0.412409171731452, 0.3925752997352563, 0.4088250297994045] + \
 [0.4116558874679148, 0.401529669005535, 0.40270344105350453, 0.40429595443532573, 0.40073388818826794, 
  0.40253056974550294, 0.40944832778266993, 0.4043074249630694, 0.40895367076311173, 0.39635875331174053]

In [61]:
np.mean(r2_list)

0.4043281936561307

In [62]:
np.max(r2_list)

0.41689655915741264

In [63]:
np.min(r2_list)

0.3917029236852765

In [64]:
np.std(r2_list)

0.005116333480914738

In [71]:
r2_young_list = [0.2979886627866627, 0.29791966173551165, 0.29894049798372246, 0.2979187079704628, 0.29677522905717924, 
                 0.2982929078460792, 0.297470635886913, 0.2981276880765539, 0.29752941801220034, 0.2974326126343281] + \
 [0.29775737342325503, 0.2971685917587894, 0.29661542064488366, 0.29805361203423697, 0.29696388859773426, 
  0.2980296619614804, 0.29759947376074625, 0.29796374504368384, 0.2987378602746452, 0.29877520216772013] + \
 [0.29729368126816813, 0.297536875365744, 0.29712291386660594, 0.29826178976730044, 0.29897802431203147, 
  0.29738676691912913, 0.29844830096340613, 0.29810847280860675, 0.2971130618823181, 0.29784913516725675] + \
 [0.2978364600790693, 0.2984750216922194, 0.29791834022461106, 0.29867866068768456, 0.29763748907401644, 
  0.29737031345668963, 0.2971455329821858, 0.29851041775624576, 0.29769424566636027, 0.2985625761694005] + \
 [0.29806662001556816, 0.29876330178642463, 0.29799405673244994, 0.29818741935243154, 0.29837500194857847, 
  0.29694199553296796, 0.297524168270463, 0.29775651985336893, 0.2973034946698915, 0.2975119348139421] + \
 [0.29831755436697216, 0.2986480671720749, 0.2979229394912053, 0.29769345004616, 0.29763799509672395, 
  0.29764273560092824, 0.2973852351966697, 0.2988492536628663, 0.2969204612587506, 0.29759043380458716] + \
 [0.29823962839056595, 0.29779789404134405, 0.29696176158748144, 0.29809748478001175, 0.2986738355270594, 
  0.29830576866608494, 0.2983491516326571, 0.2981400946086149, 0.2969641061355899, 0.29783316881224087] + \
 [0.2979634466511909, 0.2972466294400471, 0.29783690892099624, 0.29870851529671083, 0.29722774321567, 
  0.29818494613530233, 0.2979558799224957, 0.2981656108678914, 0.2978499673561832, 0.29724341026371237] + \
 [0.2966392011855652, 0.29838084246705476, 0.2977941406516834, 0.2990925401346286, 0.298092097701393, 
  0.2966685346421617, 0.29800093247416193, 0.29846895129281303, 0.29863871504967476, 0.29807356894444703] + \
 [0.2975809724615126, 0.2976094559449822, 0.2978286763471839, 0.2988616300920348, 0.29836236009760086,
  0.2980646207518681, 0.29830752658634774, 0.2973477029548681, 0.2983978591198403, 0.29669645433706526]

In [72]:
np.mean(r2_young_list)

0.2978767430592761

In [73]:
np.max(r2_young_list)

0.2990925401346286

In [74]:
np.min(r2_young_list)

0.29661542064488366

In [75]:
np.std(r2_young_list)

0.0005848193347995549

In [76]:
r2_middle_list = [0.3763524808835307, 0.3764913869783597, 0.37686628596614913, 0.3762348644905381, 0.3757916163520967, 
                  0.3764627504577214, 0.37614462200025434, 0.3767538282211552, 0.3767616103822383, 0.37620457769262705] + \
 [0.3761552796883785, 0.3759316117920193, 0.3760354090791189, 0.3767300880432346, 0.37624361808817386,
  0.37669262315645446, 0.3759886362087319, 0.37616878794175546, 0.3765844649155856, 0.37671128068397153] + \
 [0.37599501044835837, 0.37665417523377, 0.37585483127406294, 0.3765977340012482, 0.37654130439881106, 
  0.37664304683476124, 0.3767202119914149, 0.3768583334103438, 0.3756492659822931, 0.3759589405156306] + \
 [0.37661903425944254, 0.3767968688829074, 0.3760992218923751, 0.3771609885123849, 0.376448552854037, 
  0.37632361835307704, 0.37598405103166577, 0.3765507592685653, 0.3764789526033614, 0.3762910807927409] + \
 [0.3762611702312093, 0.3763197480448546, 0.37649677878644194, 0.37662994799466, 0.37665008163956504, 
  0.37637210497202, 0.37624835797642986, 0.37622432034227216, 0.37599561371821455, 0.3763029850419287] + \
 [0.37653300406965806, 0.3759039737243972, 0.3763969983312272, 0.376573049745081, 0.3766500522903836, 
  0.37603904069475236, 0.3762046041265321, 0.3771628142584512, 0.3760614760270845, 0.3764088451726335] + \
 [0.37621706863402704, 0.37645471840862466, 0.3761665456090624, 0.37632401389610515, 0.3765490176834113, 
  0.3766392842830142, 0.37713913041483693, 0.37635378769871286, 0.37645710363038964, 0.37592134717616865] + \
 [0.3763174132882995, 0.3760908824175907, 0.37674112387291603, 0.37662217899169914, 0.37639077379299135, 
  0.37668400819286885, 0.3762116865363039, 0.37619818028921903, 0.37658207088985396, 0.376324363279942] + \
 [0.37614640519389664, 0.37628080014923837, 0.37635449591510284, 0.3764068107216483, 0.37670735610182127,
  0.3760176041082308, 0.3762862995923456, 0.3767730770681694, 0.3768694190153754, 0.37635512009546523] + \
 [0.37579769211717473, 0.37646938386029394, 0.3764159135782851, 0.3766548432634974, 0.37690986830020345, 
  0.37632081354113167, 0.3763301509347212, 0.37700168613803764, 0.37637135863866855, 0.37597347487743715]

In [81]:
np.mean(r2_middle_list)

0.37639794044947933

In [82]:
np.max(r2_middle_list)

0.3771628142584512

In [83]:
np.min(r2_middle_list)

0.3756492659822931

In [84]:
np.std(r2_middle_list)

0.00031224479335830124

### Old Model Young Prediction

Prediction is average of 10 fold * 10 times

In [105]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
young_preidiction_df = pd.DataFrame(np.array(y_prediction_list).T, columns = column_list)

In [106]:
young_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1562723.0,5.0,5.130018,5.336755,5.091726,5.274494,5.314049,5.383406,5.129823,5.409705,...,5.105457,5.025791,5.727588,5.204990,5.177685,4.965321,5.130508,5.036926,5.437913,5.223881
1,708695.0,4.0,4.245168,4.633191,4.264570,4.320512,4.789810,4.134440,4.290300,4.251861,...,4.563736,4.502735,4.294883,4.338293,4.047647,4.342415,4.021832,4.729479,4.650709,4.248056
2,1828455.0,2.0,7.894469,7.539904,7.540966,7.712207,7.757643,7.667683,7.324636,7.641912,...,7.454721,7.252598,7.961985,7.541924,7.287425,7.695890,7.644617,7.860868,7.689695,7.862912
3,968859.0,8.0,5.585791,5.427691,4.593238,5.016794,5.578860,5.049002,5.689332,5.090939,...,5.420019,5.259854,5.673426,4.940588,5.127910,5.410072,5.656180,4.826082,5.279181,5.019331
4,2375401.0,5.0,4.951559,5.513785,5.798038,5.627521,5.232227,5.459647,5.578069,5.670605,...,5.254714,5.598813,5.260324,5.507310,5.595958,6.077547,5.924397,5.143947,5.399024,5.561028


In [107]:
young_preidiction_df['Predict_y'] = young_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [108]:
young_preidiction_df['Predict_y'].head()

0    5.278725
1    4.387673
2    7.615552
3    5.315835
4    5.489779
Name: Predict_y, dtype: float64

In [109]:
young_preidiction_df = young_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [110]:
young_preidiction_df['index'] = young_preidiction_df['index'].astype(int)

In [111]:
young_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1562723,5.0,5.278725
1,708695,4.0,4.387673
2,1828455,2.0,7.615552
3,968859,8.0,5.315835
4,2375401,5.0,5.489779
5,723699,6.0,6.750625
6,2069375,2.0,3.658306
7,1655669,10.0,4.659057
8,776224,7.0,5.861314
9,1002429,5.0,6.744498


In [112]:
young_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbOldModelYoungPrediction_v1.parquet'))

### Old Model Middle Prediction

In [113]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
middle_preidiction_df = pd.DataFrame(np.array(m_prediction_list).T, columns = column_list)

In [114]:
middle_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1469152.0,7.0,7.076150,7.218330,7.328291,7.326531,7.352338,7.256581,7.250031,7.032497,...,7.318269,7.198857,7.396550,7.150467,7.426195,7.134408,7.071593,7.148722,7.152225,7.252667
1,831522.0,5.0,2.990256,2.999151,2.863636,2.745602,3.034600,2.815908,3.074700,2.828677,...,2.698011,2.860187,2.814734,3.163394,2.842949,3.140445,2.564129,3.098267,3.196708,2.835105
2,1938251.0,2.0,4.235611,4.247810,4.341072,4.035480,4.044701,4.257991,4.239149,3.975514,...,3.969376,3.777669,3.835070,3.987470,4.083126,4.101276,3.864651,4.118773,3.849860,4.073575
3,1836183.0,10.0,7.657158,7.389333,7.439538,7.389019,7.533093,7.715889,7.451866,7.697068,...,7.555080,7.736094,7.832221,7.683749,7.477293,7.506588,7.514276,7.494143,7.531126,7.770050
4,1653464.0,3.0,6.398757,6.645863,6.778865,6.545725,6.431090,6.415228,6.565426,6.641722,...,6.369149,6.406950,6.345706,6.674747,6.547424,6.974433,6.861008,6.511042,6.765471,6.525588


In [115]:
middle_preidiction_df['Predict_y'] = middle_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [116]:
middle_preidiction_df['Predict_y'].head()

0    7.219572
1    2.934331
2    4.079315
3    7.539057
4    6.571742
Name: Predict_y, dtype: float64

In [117]:
middle_preidiction_df = middle_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [118]:
middle_preidiction_df['index'] = middle_preidiction_df['index'].astype(int)

In [119]:
middle_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1469152,7.0,7.219572
1,831522,5.0,2.934331
2,1938251,2.0,4.079315
3,1836183,10.0,7.539057
4,1653464,3.0,6.571742
5,2335499,7.0,7.705797
6,1538044,8.0,8.354906
7,1461202,9.0,8.299133
8,727466,5.0,5.069509
9,938061,3.0,3.236902


In [120]:
middle_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbOldModelMiddlePrediction_v1.parquet'))

### Old Model Old Prediction

In [121]:
merged_old_prediction_df = None
for old_prediction_df in fold_predictions_array_list:
    old_prediction_df['index'] = old_prediction_df['index'].astype(int)
    old_prediction_df['Real_y'] = old_prediction_df['Real_y'].astype(int)
    print(old_prediction_df.columns)
    if merged_old_prediction_df is None:
        merged_old_prediction_df = old_prediction_df
    else:
        merged_old_prediction_df = pd.merge(merged_old_prediction_df, old_prediction_df, on=['index', 'Real_y'])

Index(['index', 'Real_y', 'Predict_y_0'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_1'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_2'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_3'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_4'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_5'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_6'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_7'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_8'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_9'], dtype='object')


In [122]:
merged_old_prediction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,Predict_y_8,Predict_y_9
0,1618415,8,3.585752,3.693771,4.180027,4.032078,4.005493,3.820362,3.741325,3.814029,3.508904,4.053596
1,1688015,8,7.240756,6.952606,7.001698,6.948921,7.262212,6.759714,6.777848,7.081533,7.268421,6.793781
2,1726042,7,6.794312,6.651580,6.306438,6.236263,6.613828,6.567781,6.460809,6.347579,6.386783,6.346259
3,2461182,3,6.433945,6.381985,6.573924,6.209233,6.257556,6.185764,6.209081,6.537595,6.569862,6.445984
4,2308488,8,8.156580,8.042215,8.272901,8.220867,8.360530,8.486036,8.257619,8.316249,8.085272,8.037689


In [123]:
merged_old_prediction_df['Predict_y'] = merged_old_prediction_df.iloc[:,2:13].mean(axis = 1)

In [124]:
merged_old_prediction_df = merged_old_prediction_df[['index', 'Real_y', 'Predict_y']]

In [125]:
merged_old_prediction_df.head(10)

,index,Real_y,Predict_y
0,1618415,8,3.843534
1,1688015,8,7.008749
2,1726042,7,6.471163
3,2461182,3,6.380493
4,2308488,8,8.223596
5,2520496,5,4.702780
6,2587628,0,6.420927
7,2580220,4,4.289569
8,818170,8,8.098807
9,2568275,8,5.448313


In [126]:
merged_old_prediction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbOldModelOldPrediction_v1.parquet'))